In [1]:
%matplotlib inline
import scipy.io
import cv2
import matplotlib.pyplot as plt
import numpy as np
import math

In [2]:
hand_dataset = '/home/nauris/Dropbox/coding/Valopaat/data/hand_dataset/'

In [3]:
import glob
import os

In [4]:
im_size = 250

dataset_path = glob.glob(hand_dataset + 'test_dataset/images/*.jpg')
dataset_path += glob.glob(hand_dataset + 'training_dataset/images/*.jpg')
dataset_path += glob.glob(hand_dataset + 'validation_dataset/images/*.jpg')

print(len(dataset_path))

5628


In [5]:
def get_hand(img, coords, plot=False):
    hand_h = 26
    hand_w = 18
    
    hand = img[coords[1] - hand_h//2: coords[1] + hand_h//2,
               coords[0] - hand_w//2: coords[0] + hand_w//2]
    
    # print(hand.shape)
    
    if hand.shape[0] != hand_h or hand.shape[1] != hand_w:
        return None
    
    plt.imshow(hand)
    plt.show()

    return hand

In [10]:
def parse_annotation(annot):
    persons = []
    for person in annot['boxes']:
        # print('Number of hands', person.shape[0])
        
        if person.shape[0] > 2:
            # skip if more than two hands because we are only
            # interesting for single person hand detection
            return None
        
        label = [0, 0, 0 ,0]
        for hand in person:
            try:
                hand_label = hand[0][0][4][0] # Left or right
            except IndexError:
                return None # No hand label available
            bbox = np.array([[hand[0][0][0][0][1], hand[0][0][0][0][0]],
                             [hand[0][0][1][0][1], hand[0][0][1][0][0]],
                             [hand[0][0][2][0][1], hand[0][0][2][0][0]],
                             [hand[0][0][3][0][1], hand[0][0][3][0][0]]], np.int32)

            _xcenter = int((bbox[:, 0].max() + bbox[:, 0].min()) / 2)
            _ycenter = int((bbox[:, 1].max() + bbox[:, 1].min()) / 2)
            if hand_label == 'L':
                label[0] = _xcenter
                label[1] = _ycenter
            else:
                label[2] = _xcenter
                label[3] = _ycenter

            # print(hand_label, bbox, bbox.shape)

            # cv2.drawContours(img, [bbox], 0, (0,255,0), 3)

        # print(hstore0)
        # print(hstore1)
        # cv2.drawContours(img, [bbox], 0, (0,0,255), 3)
    return label

In [ ]:
def parse_annotations(annot):
    bboxs = []
    for person in annot['boxes']:
        # print('Number of hands', person.shape[0])
        
        # if person.shape[0] > 2:
            # skip if more than two hands because we are only
            # interesting for single person hand detection
        #     return None
        
        # label = [0, 0, 0 ,0]
        for hand in person:
            try:
                hand_label = hand[0][0][4][0] # Left or right
            except IndexError:
                return None # No hand label available
            bbox = np.array([[hand[0][0][0][0][1], hand[0][0][0][0][0]],
                             [hand[0][0][1][0][1], hand[0][0][1][0][0]],
                             [hand[0][0][2][0][1], hand[0][0][2][0][0]],
                             [hand[0][0][3][0][1], hand[0][0][3][0][0]]], np.int32)

            # _xcenter = int((bbox[:, 0].max() + bbox[:, 0].min()) / 2)
            # _ycenter = int((bbox[:, 1].max() + bbox[:, 1].min()) / 2)
            # if hand_label == 'L':
            #     label[0] = _xcenter
            #     label[1] = _ycenter
            # else:
            #     label[2] = _xcenter
            #     label[3] = _ycenter

            # print(hand_label, bbox, bbox.shape)

            # cv2.drawContours(img, [bbox], 0, (0,255,0), 3)
            bboxs.append(bbox)

        # print(hstore0)
        # print(hstore1)
        # cv2.drawContours(img, [bbox], 0, (0,0,255), 3)
    return bboxs

In [11]:
labels = []
images = []

for path in dataset_path:
    dir_path = '/'.join(os.path.dirname(path).split('/')[:-1])
    
    annot_file = dir_path + '/annotations/' + ''.join(os.path.basename(path).split('.')[:-1])
    annot = scipy.io.loadmat(annot_file + '.mat')
    
    label = parse_annotation(annot)
    # print(label)
    
    if label != None:
        
        im_file = dir_path + '/images/' + ''.join(os.path.basename(path).split('.')[:-1])
        img = cv2.imread(im_file + '.jpg')
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        if img.shape[0] >= img.shape[1]: # Image is vertically positioned

            im_w = int(img.shape[1] * im_size / img.shape[0])
            
            label[1] = int(label[1] * (im_size / img.shape[0]))
            label[3] = int(label[3] * (im_size / img.shape[0]))
            
            label[0] = int(label[0] * (im_w / img.shape[1]))
            label[2] = int(label[2] * (im_w / img.shape[1]))

            img = cv2.resize(img, (im_w, im_size))
            img = np.pad(img, ((0, 0), (0, im_size - im_w), (0, 0)),
                         'constant', constant_values=0)

        else:
            im_h = int(img.shape[0] * im_size / img.shape[1])
            
            label[1] = int(label[1] * (im_h / img.shape[0]))
            label[3] = int(label[3] * (im_h / img.shape[0]))
            
            label[0] = int(label[0] * (im_size / img.shape[1]))
            label[2] = int(label[2] * (im_size / img.shape[1]))
            
            img = cv2.resize(img, (im_size, im_h))
            img = np.pad(img, ((0, im_size - im_h), (0, 0), (0, 0)),
                         'constant', constant_values=0)
        
        # plt.imshow(img)
        # plt.show()
        
        # print('Image', img.shape, img.min(), img.mean(), img.max())
        # print(label)
        
        # hand_0 = get_hand(img, [label[0], label[1]], plot=True)
        # hand_1 = get_hand(img, [label[2], label[3]], plot=True)
        
        img = (img - 127.5) / 255
        label = np.asarray(label) / im_size
        
        images.append(img)
        labels.append(label)

images = np.asarray(images, np.float32)
labels = np.asarray(labels)

print(images.shape, labels.shape)

(3443, 250, 250, 3) (3443, 4)


In [13]:
np.save('data/hand_dataset/images.npy', images)
np.save('data/hand_dataset/labels.npy', labels)